In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import tensorflow_addons as tfa
import glob
import wandb
from wandb.keras import WandbCallback
import shutil
import matplotlib.pyplot as plt
import numpy as np

/home/lorenzoquerol/miniconda3/envs/wssv-recognition/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
EXPERIMENT_NAME = "efficientnetv2b0-average-runs"
DATA_DIR = "dataset-augmented"
AUTOTUNE = tf.data.AUTOTUNE
CONFIG = dict(
    epochs=100,
    learning_rate=1e-4,
    batch_size=64,
    img_shape=(224, 224),
    input_shape=(224, 224, 3),
    num_classes=2,
    dropout_rate=0.5,
    es_patience=10,
    seed_value=42,
)

In [3]:
def train():
    image_count = len(glob.glob(f"{DATA_DIR}/*/*/*.jpg"))
    healthy_count = len(glob.glob(f"{DATA_DIR}/train/healthy/*.jpg"))
    wssv_count = len(glob.glob(f"{DATA_DIR}/train/wssv/*.jpg"))

    train_dir = os.path.join(DATA_DIR, "train")
    valid_dir = os.path.join(DATA_DIR, "val")
    test_dir = os.path.join(DATA_DIR, "test")

    train_set = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        seed=CONFIG["seed_value"],
        image_size=CONFIG["img_shape"],
        batch_size=CONFIG["batch_size"],
        label_mode="categorical",
    ).prefetch(buffer_size=AUTOTUNE)

    valid_set = tf.keras.utils.image_dataset_from_directory(
        valid_dir,
        seed=CONFIG["seed_value"],
        image_size=CONFIG["img_shape"],
        batch_size=CONFIG["batch_size"],
        label_mode="categorical",
    ).prefetch(buffer_size=AUTOTUNE)

    test_set = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        seed=CONFIG["seed_value"],
        image_size=CONFIG["img_shape"],
        batch_size=CONFIG["batch_size"],
        label_mode="categorical",
    ).prefetch(buffer_size=AUTOTUNE)

    base_model = tf.keras.applications.EfficientNetV2B0(
        input_shape=CONFIG["input_shape"],
        include_top=False,
        classes=CONFIG["num_classes"],
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=CONFIG["input_shape"])
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(rate=CONFIG["dropout_rate"], seed=CONFIG["seed_value"])(
        x
    )
    outputs = tf.keras.layers.Dense(units=CONFIG["num_classes"], activation="softmax")(
        x
    )

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG["learning_rate"]),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
            tfa.metrics.F1Score(
                num_classes=CONFIG["num_classes"],
                average="weighted",
                name="f1_score",
                threshold=0.5,
            ),
            tf.keras.metrics.FalseNegatives(name="false_negatives"),
            tf.keras.metrics.TruePositives(name="true_positives"),
            tf.keras.metrics.FalsePositives(name="false_positives"),
            tf.keras.metrics.TrueNegatives(name="true_negatives"),
        ],
    )

    class_0_weight = (1 / healthy_count) * (image_count / 2.0)
    class_1_weight = (1 / wssv_count) * (image_count / 2.0)
    class_weight = {0: class_0_weight, 1: class_1_weight}

    if os.path.exists("checkpoints_efficientnetv2b0"):
        shutil.rmtree("checkpoints_efficientnetv2b0")
        os.makedirs("checkpoints_efficientnetv2b0")

    checkpoint_path = "checkpoints_efficientnetv2b0/cp-{epoch:04d}.ckpt"

    model.save_weights(checkpoint_path.format(epoch=0))

    model.fit(
        train_set,
        epochs=CONFIG["epochs"],
        validation_data=valid_set,
        class_weight=class_weight,
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                save_weights_only=True,
                monitor="val_loss",
                mode="min",
                verbose=1,
                save_best_only=True,
            ),
            WandbCallback(save_model=False),
        ],
    )

In [4]:
for i in range(5):
    wandb.init(
        project="wssv-recognition",
        config=CONFIG,
        group=EXPERIMENT_NAME,
        job_type="train",
    )
    train()
    wandb.finish()

wandb: Currently logged in as: renzo_querol. Use `wandb login --relogin` to force relogin


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 0.8386 - precision: 0.5679 - recall: 0.5679 - f1_score: 0.5899 - false_negatives: 442.0000 - true_positives: 581.0000 - false_positives: 442.0000 - true_negatives: 581.0000
Epoch 1: val_loss improved from inf to 0.66560, saving model to checkpoints_efficientnetv2b0/cp-0001.ckpt
16/16 [==============================] - 15s 318ms/step - loss: 0.8386 - precision: 0.5679 - recall: 0.5679 - f1_score: 0.5899 - false_negatives: 442.0000 - true_positives: 581.0000 - false_positives: 442.0000 - true_negatives: 581.0000 - val_loss: 0.6656 - val_precision: 0.5714 - val_recall: 0.5714 - val_f1_score: 0.5650 - val_false_negatives: 21.0000 - val_true_positives: 28.0000 - val_false_positives: 21.0000 - val_true_negatives: 28.0000
Epoch 2/100
16/16 [==============================] - ETA: 0s - loss: 0.8018 - precision

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▂▂▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇█▇▇▇▇▇██▇██▇██████████
false_negatives,█▇▇▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▁▂▂▂▂▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁
false_positives,█▇▇▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▁▂▂▂▂▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▅▅▅▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
precision,▁▂▂▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇█▇▇▇▇▇██▇██▇██████████
recall,▁▂▂▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇█▇▇▇▇▇██▇██▇██████████
true_negatives,▁▂▂▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇█▇▇▇▇▇██▇██▇██████████
true_positives,▁▂▂▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇█▇▇▇▇▇██▇██▇██████████
val_f1_score,▁▂▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇█▇█████████████████████
val_false_negatives,█▇▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 0.7729 - precision: 0.5885 - recall: 0.5885 - f1_score: 0.6151 - false_negatives: 421.0000 - true_positives: 602.0000 - false_positives: 421.0000 - true_negatives: 602.0000
Epoch 1: val_loss improved from inf to 0.69140, saving model to checkpoints_efficientnetv2b0/cp-0001.ckpt
16/16 [==============================] - 9s 185ms/step - loss: 0.7729 - precision: 0.5885 - recall: 0.5885 - f1_score: 0.6151 - false_negatives: 421.0000 - true_positives: 602.0000 - false_positives: 421.0000 - true_negatives: 602.0000 - val_loss: 0.6914 - val_precision: 0.5510 - val_recall: 0.5510 - val_f1_score: 0.5799 - val_false_negatives: 22.0000 - val_true_positives: 27.0000 - val_false_positives: 22.0000 - val_true_negatives: 27.0000
Epoch 2/100
16/16 [==============================] - ETA: 0s - loss: 0.7427 - precision:

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███▇█▇▇▇█████████
false_negatives,█▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁
false_positives,█▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁
loss,█▇▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁
precision,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███▇█▇█▇█████████
recall,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███▇█▇█▇█████████
true_negatives,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███▇█▇█▇█████████
true_positives,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███▇█▇█▇█████████
val_f1_score,▁▃▅▆▇▇▇▇▇▇▇▇████████████████████████████
val_false_negatives,█▆▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▂▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█▇▇▇████████████████
false_negatives,█▇▅▅▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
false_positives,█▇▅▅▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
precision,▁▂▄▄▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇▇████████████████
recall,▁▂▄▄▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇▇████████████████
true_negatives,▁▂▄▄▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇▇████████████████
true_positives,▁▂▄▄▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇▇████████████████
val_f1_score,▁▅▇▇██▇▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val_false_negatives,█▄▂▂▁▁▂▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 0.8056 - precision: 0.4966 - recall: 0.4966 - f1_score: 0.5298 - false_negatives: 515.0000 - true_positives: 508.0000 - false_positives: 515.0000 - true_negatives: 508.0000
Epoch 1: val_loss improved from inf to 0.54414, saving model to checkpoints_efficientnetv2b0/cp-0001.ckpt
16/16 [==============================] - 8s 203ms/step - loss: 0.8056 - precision: 0.4966 - recall: 0.4966 - f1_score: 0.5298 - false_negatives: 515.0000 - true_positives: 508.0000 - false_positives: 515.0000 - true_negatives: 508.0000 - val_loss: 0.5441 - val_precision: 0.7755 - val_recall: 0.7755 - val_f1_score: 0.7906 - val_false_negatives: 11.0000 - val_true_positives: 38.0000 - val_false_positives: 11.0000 - val_true_negatives: 38.0000
Epoch 2/100
16/16 [==============================] - ETA: 0s - loss: 0.7576 - precision:

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇██████████████
false_negatives,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
false_positives,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
precision,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇██████████████
recall,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇██████████████
true_negatives,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇██████████████
true_positives,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇██████████████
val_f1_score,▁▇▇▇██▇▇▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
val_false_negatives,█▂▂▂▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 0.8494 - precision: 0.4272 - recall: 0.4272 - f1_score: 0.4493 - false_negatives: 586.0000 - true_positives: 437.0000 - false_positives: 586.0000 - true_negatives: 437.0000
Epoch 1: val_loss improved from inf to 0.82902, saving model to checkpoints_efficientnetv2b0/cp-0001.ckpt
16/16 [==============================] - 9s 223ms/step - loss: 0.8494 - precision: 0.4272 - recall: 0.4272 - f1_score: 0.4493 - false_negatives: 586.0000 - true_positives: 437.0000 - false_positives: 586.0000 - true_negatives: 437.0000 - val_loss: 0.8290 - val_precision: 0.4082 - val_recall: 0.4082 - val_f1_score: 0.4082 - val_false_negatives: 29.0000 - val_true_positives: 20.0000 - val_false_positives: 29.0000 - val_true_negatives: 20.0000
Epoch 2/100
16/16 [==============================] - ETA: 0s - loss: 0.7883 - precision:

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████
false_negatives,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
false_positives,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████
recall,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████
true_negatives,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████
true_positives,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████
val_f1_score,▁▅▅▅▆▇▇▇▇▇▇▇▇███████████████████████████
val_false_negatives,█▅▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
